**Full autonomous agent loop**

Using LangChain’s PydanticOutputParser with a Pydantic schema. This ensures the LLM output is always parsed into structured JSON safely, avoiding crashes from extra text.

Included a book_flight API tool, so you see multi-tool orchestration (HR policy lookup + email + Slack + flight booking).

Agent can now chain across multiple tools in one run:
Look up HR → Email → Slack → Flight → Finish.

**Agent memory persists between runs in Colab.**

That way:

*   If run the agent once, it will book a flight and send email.
*   If you run it again, it will remember that it already booked the flight or emailed the manager.

We’ll do this with a JSON file as persistent memory storage.

Added a **Memory Inspector** so you can peek into what the agent has done across runs.

In [1]:
!pip install langchain langchain-openai faiss-cpu openai
!pip install -U langchain-community
!pip install pydantic==2.11.0
!pip install requests==2.32.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.6/442.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━

In [2]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_APIKEY")

In [3]:
# Load documents & create vector store (RAG setup)
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS # Import FAISS from langchain_community
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document
from langchain.chains import RetrievalQA
from langchain.agents import initialize_agent, AgentType, tool

In [4]:
# Example HR policy text
hr_policy = """
Employees are entitled to 20 paid leave days per year.
Unused leaves cannot be carried forward.
Sick leave requires a doctor’s certificate if more than 3 days.
"""

# Split into chunks
splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=20)
docs = splitter.split_documents([Document(page_content=hr_policy)])

# Create embeddings + vector store
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = FAISS.from_documents(docs, embeddings)

# Create retriever for RAG
retriever = vectorstore.as_retriever()

**Define Tools**

In [5]:
# Email Tool
@tool
def send_email(to: str, subject: str, body: str):
    """Send an email to the given address with subject and body."""
    print(f"📧 Email sent to {to}\nSubject: {subject}\nBody: {body}")
    return "Email sent successfully."

In [6]:
# Slack Notification Tool
@tool
def notify_slack(channel: str, message: str):
    """Send a Slack notification to a given channel."""
    print(f"💬 Slack message to {channel}: {message}")
    return "Slack notification sent."

In [7]:
@tool
def hr_policy_lookup(query: str):
    """Look up HR policy details from company documents."""
    return rag_chain.run(query)

In [8]:
# Prompt the LLM to return JSON
goal = "Summarize HR leave policy, email it to manager, and confirm in Slack"
memory = [
    {"action": {"action": "hr_policy_lookup", "args": {"query": "Summarize HR leave policy"}},
     "result": "Employees get 20 paid leave days. Sick leave >3 days needs a certificate. No carry forward."},
    {"action": {"action": "send_email", "args": {"to": "manager@example.com", "subject": "HR Leave Policy Summary", "body": "..."}},
     "result": "Email sent successfully."}
]

In [18]:
# Flight Booking Tool
@tool
def book_flight(from_city: str, to_city: str, date: str, budget: int):
    """Book a flight given origin, destination, date, and budget."""
    # Fake database of flights
    flights = [
        {"airline": "AirOne", "price": 250, "from": "New York", "to": "Chicago"},
        {"airline": "SkyJet", "price": 320, "from": "New York", "to": "Chicago"},
        {"airline": "BudgetAir", "price": 180, "from": "Boston", "to": "Chicago"},
        {"airline": "BudgetAir", "price": 250, "from": "Boston", "to": "Chicago"}
    ]
    options = [f for f in flights if f["from"] == from_city and f["to"] == to_city and f["price"] <= budget]
    if not options:
        return f"No flights found under ${budget} from {from_city} to {to_city} on {date}."
    chosen = options[0]
    return f"Booked {chosen['airline']} flight from {from_city} to {to_city} on {date} for ${chosen['price']}."

In [10]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.agents import initialize_agent, AgentType

# Build Agent with Orchestration
# Setup LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# RAG Chain (for grounding answers)
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever
)

# **🔑 What Changed**


*   Added agent_memory.json file.
*   On each run:
    *   Loads memory from previous runs.
    *   Adds new actions/results.
    *   Saves updated memory back to file.
*   So if you rerun the cell, the agent remembers past steps.

In [11]:
# -----------------------------
# Memory Inspector
# -----------------------------
def inspect_memory(filter_action: str = None):
    memory = load_memory()
    if not memory:
        print("⚠️ Memory is empty.")
        return

    if filter_action:
        filtered = [m for m in memory if m["action"]["action"] == filter_action]
        if not filtered:
            print(f"⚠️ No records found for action: {filter_action}")
            return
        print(f"🔍 Showing {len(filtered)} record(s) for '{filter_action}':")
        for m in filtered:
            print(json.dumps(m, indent=2))
    else:
        print("📖 Full memory:")
        print(json.dumps(memory, indent=2))


In [19]:
import time
import json
from typing import Optional, Dict, List
from pydantic import BaseModel
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate

# -----------------------------
# 1. Define Schema with Pydantic
# -----------------------------
class AgentAction(BaseModel):
    action: str  # e.g., hr_policy_lookup, send_email, notify_slack, FINISH
    args: Optional[Dict] = {}

parser = PydanticOutputParser(pydantic_object=AgentAction)

# -----------------------------
# 2. Prompt Template
# -----------------------------
prompt = PromptTemplate(
    template="""
You are an autonomous agent.
Your goal: {goal}
Memory so far: {memory}

Decide the NEXT ACTION to take.

Available actions:
- hr_policy_lookup(query: str)
- send_email(to: str, subject: str, body: str)
- notify_slack(channel: str, message: str)
- book_flight(from_city: str, to_city: str, date: str, budget: int)
- FINISH

{format_instructions}
""",
    input_variables=["goal", "memory"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# -----------------------------
# 3. Persistent Memory Handling
# -----------------------------
MEMORY_FILE = "agent_memory.json"

def load_memory() -> List[Dict]:
    if os.path.exists(MEMORY_FILE):
        with open(MEMORY_FILE, "r") as f:
            return json.load(f)
    return []

def save_memory(memory: List[Dict]):
    with open(MEMORY_FILE, "w") as f:
        json.dump(memory, f, indent=2)

# -----------------------------
# 4. Agent Loop
# -----------------------------
def autonomous_agent_structured(goal, max_steps=5):
    memory = load_memory()

    for step in range(max_steps):
        print(f"\n🌀 Step {step+1} ------------------------")

        # Fill prompt
        filled_prompt = prompt.format(goal=goal, memory=memory)
        print("📜 FULL PROMPT SENT TO LLM:\n", filled_prompt)

        # Get LLM output
        llm_output = llm.invoke(filled_prompt).content
        print("📝 RAW LLM OUTPUT:", llm_output)

         # Parse with Pydantic parser
        try:
            agent_action = parser.parse(llm_output)
        except Exception as e:
            print("❌ Parsing failed:", e)
            agent_action = AgentAction(action="FINISH", args={})

        print("✅ PARSED ACTION:", agent_action)

        # -----------------------------
        # 5. Execute Tool
        # -----------------------------
        if agent_action.action == "FINISH":
            print("🎯 Goal achieved.")
            break
        elif agent_action.action == "hr_policy_lookup":
            result = hr_policy_lookup.invoke(agent_action.args)
        elif agent_action.action == "send_email":
            result = send_email.invoke(agent_action.args)
        elif agent_action.action == "notify_slack":
            result = notify_slack.invoke(agent_action.args)
        elif agent_action.action == "book_flight":
            result = book_flight.invoke(agent_action.args)
        else:
            result = "Unknown action."

        print("📌 Result:", result)

        # Update memory
        memory.append({"action": agent_action.model_dump(), "result": result})
        save_memory(memory)
        time.sleep(2)

# -----------------------------
# 6. Run Example
# -----------------------------
autonomous_agent_structured(
    "Summarize HR leave policy, email it to manager, confirm in Slack, and also book a flight from Boston to Chicago for Sept 15 with a budget of $400"
)

# Inspect memory
print("\n====== FULL MEMORY ======")
inspect_memory()

print("\n====== FILTER: Booked Flights ======")
inspect_memory("book_flight")


🌀 Step 1 ------------------------
📜 FULL PROMPT SENT TO LLM:
 
You are an autonomous agent.
Your goal: Summarize HR leave policy, email it to manager, confirm in Slack, and also book a flight from Boston to Chicago for Sept 15 with a budget of $400
Memory so far: [{'action': {'action': 'hr_policy_lookup', 'args': {'query': 'leave policy'}}, 'result': "Employees are entitled to 20 paid leave days per year. Unused leave days cannot be carried forward to the next year. If an employee takes sick leave for more than 3 days, a doctor's certificate is required."}, {'action': {'action': 'send_email', 'args': {'to': 'manager@example.com', 'subject': 'Summary of HR Leave Policy', 'body': "The HR leave policy states that employees are entitled to 20 paid leave days per year. Unused leave days cannot be carried forward to the next year. If an employee takes sick leave for more than 3 days, a doctor's certificate is required."}}, 'result': 'Email sent successfully.'}, {'action': {'action': 'notify